In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import re
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, AutoModelForSequenceClassification, Trainer, TrainingArguments, DistilBertConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
import random
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.nn import CrossEntropyLoss
import nltk
from transformers import DistilBertTokenizer
from sklearn.metrics import classification_report
from huggingface_hub import create_repo
import os
from git import Repo  # For interacting with Git
from huggingface_hub import HfApi
import shutil

In [ ]:
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git init
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git add .

!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git commit -m "Initial commit"
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git remote add origin https://github.com/eyinlojuoluwa/distilbert-base-uncased-not-usable-code.git
!cd "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code" && git push -u origin main

In [ ]:
import json
import os
import shutil
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from huggingface_hub import create_repo, login, whoami
import os
import shutil
from git import Repo
from git.exc import GitCommandError


# Load tokens from the tokens.json file
with open("tokens.json", "r") as f:
    tokens = json.load(f)

HUGGINGFACE_TOKEN = tokens["HUGGINGFACE_TOKEN"]
GITHUB_TOKEN = tokens["GITHUB_TOKEN"]

# --- Configuration ---
MODEL_DIR = "./final_model"
GITHUB_REPO_PATH = "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code"
HUGGINGFACE_REPO_NAME = "distilbert-base-uncased-not-usable"
GITHUB_COMMIT_MESSAGE = "Auto-push: Updated notebooks"
HUGGINGFACE_COMMIT_MESSAGE = "Upload fine-tuned model"

# --- Hugging Face Auth + Upload ---
login(token=HUGGINGFACE_TOKEN)
print(f"Logged in as: {whoami()['name']}")
create_repo(HUGGINGFACE_REPO_NAME, exist_ok=True)

model = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_DIR)

model.push_to_hub(HUGGINGFACE_REPO_NAME, commit_message=HUGGINGFACE_COMMIT_MESSAGE)
tokenizer.push_to_hub(HUGGINGFACE_REPO_NAME, commit_message=HUGGINGFACE_COMMIT_MESSAGE)
print("✅ Model and tokenizer pushed to Hugging Face.")

# --- GitHub Notebook File Sync ---
# Specify the exact notebook file to push (change this as needed)
notebook_to_push = "sync_tool.ipynb"  # Specify the exact file here

# Replace the file in the GitHub repo
src = os.path.join(os.getcwd(), notebook_to_push)
dst = os.path.join(GITHUB_REPO_PATH, notebook_to_push)

if os.path.exists(dst):
    os.remove(dst)
    print(f"Removed existing file: {dst}")
shutil.copy(src, dst)
print(f"Copied {notebook_to_push} to GitHub repository.")

# --- GitHub Commit & Push ---
os.chdir(GITHUB_REPO_PATH)
os.system("git init")
os.system("git add .")
os.system(f'git commit -m "{GITHUB_COMMIT_MESSAGE}"')
os.system("git branch -M main")
os.system("git remote remove origin || true")
os.system("git remote add origin https://github.com/eyinlojuoluwa/distilbert-base-uncased-not-usable-code.git")
os.system("git push -u origin main")
print("✅ Code pushed to GitHub.")


In [ ]:
import os
print(os.getcwd())

In [ ]:
ls /Users/your_username/Documents/GH_repo/distilbert-base-uncased-not-usable-code/.git

In [ ]:
import os

# Define the path
repo_path = "/Users/adekunleajibode/Documents/GH_repo/distilbert-base-uncased-not-usable-code"

# List files in the directory
files = os.listdir(repo_path)
print(files)

In [ ]:
repo_name = "distilbert-base-uncased-not-usable"  # Change this to your preferred repository name
create_repo(repo_name)

# Replace these with your actual paths
model_dir = "./final_model"

# Push model, tokenizer, and config to Hugging Face Hub
model = DistilBertForSequenceClassification.from_pretrained(model_dir)
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

In [1]:
import os

# Get the tokens from the environment variables
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
github_token = os.getenv("GITHUB_TOKEN")

# Now you can use the tokens in your code
print("Hugging Face Token:", huggingface_token)
print("GitHub Token:", github_token)

Hugging Face Token: None
GitHub Token: None


In [2]:
pwd

'/Users/adekunleajibode/Paper2'